In [1]:
import sys
import os
import time
import codecs
import pandas as pd
from pprint import pprint
import re

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")

import emolex
import paths
import tokenizer
import useful_methods as my_methods
import train_datas

### one-hot encoding

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
onehot_encoder = DictVectorizer()

In [ ]:
instances = [
    {'city': 'New York'},
    {'city': 'San Francisco'},
    {'city': 'Chapel Hill'}]

In [ ]:
onehot_encoder.fit_transform(instances).toarray()

### The bag-of-words representation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
    'UNC played Duke in basketball',
    'Duke lost the basketball game',
    'I ate a sandwich'
]

In [ ]:
# vectorizer = CountVectorizer(analyzer=tokenizer.Lemma, stop_words="english")
vectorizer = CountVectorizer(stop_words="english")

vectorizer.fit_transform(corpus).todense()

In [ ]:
vectorizer.vocabulary_

## Tunning Models with Grid Search

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [17]:
pipeline = Pipeline([
   ('vect', TfidfVectorizer(stop_words='english')),
   ('clf', LogisticRegression())
])
parameters = {
   'vect__max_df': (0.25, 0.5, 0.75),
   'vect__stop_words': ('english', None),
   'vect__max_features': (2500, 5000, 10000, None),
   'vect__ngram_range': ((1, 1), (1, 2)),
   'vect__use_idf': (True, False),
   'vect__norm': ('l1', 'l2'),
   'clf__penalty': ('l1', 'l2'),
   'clf__C': (0.01, 0.1, 1, 10),
}

In [18]:
if __name__ == "__main__":
    grid_search = GridSearchCV(
        pipeline,
        parameters,
        n_jobs=-1,
        verbose=1,
        scoring='accuracy',
        cv=3)
    
    df = pd.read_csv('data/sms.csv')
    X, y, = df['message'], df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    grid_search.fit(X_train, y_train)
    
    print('Best score: %0.3f' % grid_search.best_score_)
    print('Best parameters set:')
    best_parameters = grid_search.best_estimator_.get_params()

    for param_name in sorted(parameters.keys()):
       print('\t%s: %r' % (param_name, best_parameters[param_name]))
    predictions = grid_search.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('Precision:', precision_score(y_test, predictions))
    print('Recall:', recall_score(y_test, predictions))

OSError: File b'data/sms.csv' does not exist